## Bidding Architecture

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage, AIMessage
from langchain.chat_models import ChatOpenAI
from langgraph.checkpoint.sqlite import SqliteSaver

from langgraph.graph import END, StateGraph, MessageGraph

import functools
import operator
from typing import List, Sequence, TypedDict, Annotated
import json
import os
import random

from IPython.display import Image, display

import concurrent.futures

In [3]:
unique_id = "Market Optimisation"
os.environ["LANGCHAIN_PROJECT"] = f"Tracing Walkthrough - {unique_id}"

In [4]:
# from langsmith import Client

# client = Client()

In [5]:
class PromptUpdate(BaseModel):
    """Review of the prompt"""
    updated_prompt: str = Field(description="Updated prompt based on the review")
    justification: str = Field(description="Justification for the changes made")

class Bid(BaseModel):
    """Bid value for the prompt"""
    position: str = Field(description="Position of the expert providing the bid")
    bid: int = Field(description="Bid value for the prompt")

class CorePrinciples:
    def __init__(self, core_principles: List[str]):
        self.core_principles = core_principles
    
    def add_principle(self, principle: str):
        """
        Adds a principle to the core principles list.
        
        :param principle: The principle to be added.
        """
        self.core_principles.append(principle)
        
    def __str__(self):
        """
        Returns a string representation of the core principles, each principle is listed on a new line with a preceding dash.
        
        Example:
        - principle 1
        - principle 2
        ...
        """
        return "\n".join([f"- {principle}" for principle in self.core_principles])

class ExpertAgent:
    """
    Expert Agent class defining agents that provide feedback on prompts.
    """

    def __init__(self, position: str, core_principles: CorePrinciples, temp: float = 1.0, model: str = "gpt-4o"):
        self.position = position
        self.core_principles = core_principles
        self.system_message = SystemMessage(content=f"""You are an experienced: {self.position}. Your core principles are:
{self.core_principles}""")
        self.llm = ChatOpenAI(
            temperature=temp,
            model=model,
        )

    def bid(self, prompt: str, criteria: str, history) -> float:
        """
        Bids on the prompt based on the expert's expertise.
        """
        template = """```{prompt}```

Your task is to bid on the prompt above.

The success criteria for the updated prompt are as follows:
{criteria}
Use this information to inform your bid.

The bid must reflect how much you believe the prompt needs improving in light of your core principles.

The bid must be an integer between 1 and 10. Below is a scale to guide your bid:
- 1: The prompt is perfect in light of your core principles.
- 2: The prompt is excellent in light of your core principles.
- 5: The prompt is okay but improvements are needed in light of your core principles.
- 8: The prompt is not good enough and significant improvements are needed in light of your core principles.
- 10: The prompt is terrible and wholesale changes are needed in light of your core principles.

If you are unsure, be critical rather than lenient; it is better to overbid than underbid.

Your bid process should be as follows:
1. Read the prompt carefully as an expert {position}.
2. Think carefully about how much you believe the prompt needs improving in light of your core principles.
3. Critically bid on the prompt.
4. Ensure that your bid is an integer between 1 and 10.
5. Submit your bid.

{format_instructions}

Return only your position and bid. Example output:

{{
    "position": "{position}",
    "bid": "Bid value",
}}

You will be penalized if your output cannot be parsed correctly."""
        pydantic_parser = PydanticOutputParser(pydantic_object=Bid)
        prompt_template = PromptTemplate(
            system_message=self.system_message,
            template=template,
            input_variables=["position", "prompt", "history", "criteria"],
            partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
        )
        chain = prompt_template | self.llm | pydantic_parser
        for _ in range(3):
            try:
                completion = chain.invoke({"position": self.position, "prompt": prompt, "history": history, "criteria": criteria})
                # Validate the output before returning
                if completion.position and completion.bid:
                    return completion
                # else:
                #     print("Validation failed: Missing required fields in completion")
                #     print("Raw output:", completion)
            except Exception as e:
                print("Exception occurred:", e)
                continue
        else:
            raise Exception("Failed to parse output after 3 attempts")
        
    def self_reflection_graph(self, criteria, history) -> MessageGraph:
        """
        Constructs a graph for self-reflection and improvement of prompts.
        """

        def generation_node(state: Sequence[BaseMessage], history) -> List[BaseMessage]:
            pydantic_parser = PydanticOutputParser(pydantic_object=PromptUpdate)
            prompt_text = """Your task is to improve the user's prompt in light of your core principles.
If the user provides feedback and recommendations for the prompt, respond with a revised version of your previous attempts.
DO NOT directly insert the raw feedback into the prompt. You must action the feedback, keeping in mind your core principles.
You must provide justification, in less than 100 words, for any changes you make to the prompt.

The success criteria for the updated prompt are as follows:
{criteria}
You will be penalized if your updated prompt does not meet the success criteria.

Below are strict guidelines that you MUST follow if making changes to the prompt:
- DO NOT modify existing restrictions.
- DO NOT modify or remove negations.
- DO NOT add, modify or remove placeholders denoted by curly braces. IT IS ESSENTIAL PLACEHOLDERS REMAIN UNCHANGED.
- ALWAYS treat placeholders as the actual content.
You will be penalized if you do not follow these guidelines.

Your update process should be as follows:
1. Read the prompt carefully as an expert {position}.
2. Think carefully about how you can implement the user's feedback.
3. Implement the changes to the prompt.
4. Ensure that your changes adhere to the strict guidelines. DO NOT REMOVE PLACEHOLDERS.
5. Provide justification for the changes you made.
6. Ensure that your justification is less than 100 words.
7. Submit your updated prompt and justification.

{format_instructions}

Return the updated prompt along with your justification. Example output:

{{
    "updated_prompt": "Updated prompt based on the review",
    "justification": "Justification for the changes made"
}}

You will be penalized if your output cannot be parsed correctly.
"""
            prompt = HumanMessagePromptTemplate(
                prompt = PromptTemplate(
                    system_message=self.system_message,
                    template=prompt_text,
                    input_variables=["history", "criteria", "position"],
                    partial_variables={"format_instructions": pydantic_parser.get_format_instructions()}
                )
            )
            prompt = ChatPromptTemplate.from_messages(
                [
                    prompt,
                    MessagesPlaceholder(variable_name="messages"),
                ]
            )
            for _ in range(3):
                try:
                    generate = prompt | self.llm | pydantic_parser
                    res = generate.invoke({"messages": state, "history": history, "criteria": criteria, "position": self.position})
                    updated_prompt_message = AIMessage(content=res.updated_prompt)
                    justification_message = AIMessage(content=res.justification)
                    new_state = state + [updated_prompt_message, justification_message]
                    # print(f"New state: {new_state}")
                    return new_state
                except Exception as e:
                    print("Exception occurred:", e)
                    continue
            else:
                raise Exception("Failed to parse output after 3 attempts")
            
        def reflection_node(messages: Sequence[BaseMessage]) -> List[BaseMessage]:
            reflection_prompt = ChatPromptTemplate.from_messages(
                [
                    (
                        "system",
                        f"""{self.system_message.content} 
Your task is to think outside the box and provide feedback on the user's prompt with creative recommendations on how to improve it in light of your core principles:
Your feedback must be less than 100 words so think carefully about the most critical aspects of the prompt that need improvement.

The success criteria for the updated prompt are as follows:
{criteria}
Use this information to inform your decision.

Below are strict guidelines that you MUST follow when providing feedback and recommendations:
- DO NOT suggest modifying existing restrictions.
- DO NOT suggest modifying or removing negations.
- DO NOT suggest adding, modifying or removing placeholders denoted by curly braces. IT IS ESSENTIAL PLACEHOLDERS REMAIN UNCHANGED.
- ALWAYS treat placeholders as the actual content.
You will be penalized if you do not follow these guidelines.

Your reviewal process should be as follows:
1. Read the prompt carefully as an expert {self.position}.
2. Identify the most critical aspects of the prompt that need improvement. 
3. Think outside the box to provide creative feedback with recommendations on how to improve the prompt in light of your core principles.
4. Ensure that your feedback is less than 100 words.
5. Ensure that your feedback follows the strict guidelines provided above. DO NOT SUGGEST REMOVING PLACEHOLDERS.
6. Submit your feedback.
""",
                    ),
                    MessagesPlaceholder(variable_name="messages"),
                ]
            )
            reflect = reflection_prompt | self.llm
            res = reflect.invoke({"messages": messages})
            # We treat the output of this as human feedback for the generator
            return HumanMessage(content=res.content)

        builder = MessageGraph()
        generation_node = functools.partial(generation_node, history=history)
        builder.add_node("generate", generation_node)
        builder.add_node("reflect", reflection_node)
        builder.set_entry_point("generate")

        def should_continue(state: List[BaseMessage]):
            if len(state) > 6:
                # End after 3 iterations
                return END
            return "reflect"

        builder.add_conditional_edges("generate", should_continue)
        builder.add_edge("reflect", "generate")
        graph = builder.compile()

        return graph
    
    def update_prompt(self, prompt: str, criteria: str, history) -> str:
        """
        Uses self_reflection_graph to iteratively act on feedback and update prompt
        """
        graph = self.self_reflection_graph(criteria, history)
        inputs = [HumanMessage(content=prompt)]
        updated_prompt = graph.invoke(inputs)
        return updated_prompt

In [6]:
class Experts(BaseModel):
    """Details of experts generated by the leader agent."""
    positions: List[str] = Field(description="List of positions of the experts")    
    core_principles: List[List[str]] = Field(description="Core principles of the workers in the workforce")

class ModeratorAgent:
    """
    Moderator Agent class defining an agent that builds the experts and moderates the bidding process.
    """

    def __init__(self, base_prompt: str, criteria: str = None, experts: List[ExpertAgent] = None):
        self.base_prompt = base_prompt
        self.criteria = criteria
        self.experts = experts
        self.iteration = 0

    def bidding_process(self, state: dict) -> dict:
        """
        Execution of bidding process to select the next expert to process the prompt.
        """
        self.iteration += 1
        # Collect bids from all experts
        with concurrent.futures.ThreadPoolExecutor() as executor:
            futures = [executor.submit(expert.bid, state['prompt'], self.criteria, state["messages"]) for expert in self.experts]
            bids = [future.result() for future in concurrent.futures.as_completed(futures)]
        # Sort bids in descending order
        bids.sort(key=lambda x: x.bid, reverse=True)
        for i in range(len(bids)):
            print("Position: {position}, Bid: {bid}".format(position=bids[i].position, bid=bids[i].bid))
        # If a tie occurs, randomly select a expert from the tied experts, else select the expert with the highest bid
        max_bid = max(bids, key=lambda x: x.bid).bid
        tied_experts = [expert for expert in bids if expert.bid == max_bid]
        if len(tied_experts) > 1:
            next_expert = tied_experts[random.randint(0, len(tied_experts) - 1)]
        else:
            next_expert = tied_experts[0]
        # Update the state with the next expert to process
        if max_bid <= 2.0 or self.iteration >= 10:
            return {"next": "FINISH", "prompt": state["prompt"], "messages": state["messages"]}
        else:
            return {"next": next_expert.position, "prompt": state["prompt"], "messages": state["messages"]}
        
    def construct_expert_graph(self):
        """
        Constructs a graph of expert agents based on their roles and functions.
        """

        def agent_node(state, agent, position):
            try:
                result = agent.update_prompt(state["prompt"], self.criteria, state["messages"])
                print(f"Result: {result}")
                updated_prompt = result[-2].content
                justification = result[-1].content
                return {
                    "messages": state["messages"] + [AIMessage(content=f"Updated Prompt: {updated_prompt}, Justification: {justification}", name=position)],
                    "prompt": updated_prompt,
                }
            except Exception as e:
                # Log the error and return to moderator with the most recent prompt
                print(f"Parsing failed for {position}: {e}")
                return {
                    "messages": state["messages"] + [AIMessage(content=f"Error: Parsing failed for {position} - {e}", name=position)],
                    "prompt": state["prompt"],
                    "next": "moderator"
                }
        
        # The agent state is the input to each node in the graph
        class AgentState(TypedDict):
            # The annotation tells the graph that new messages will always be added to the current states
            messages: Sequence[BaseMessage]
            next: str
            prompt: str

        workflow = StateGraph(AgentState)
        for expert in self.experts:
            # Create a node for each expert agent
            node = functools.partial(agent_node, agent=expert, position=expert.position)
            workflow.add_node(expert.position, node)
        workflow.add_node("moderator", self.bidding_process)

        members = [expert.position for expert in self.experts]
        for member in members:
            # We want our experts to ALWAYS "report back" to the moderator when done
            workflow.add_edge(member, "moderator")
        # The moderator populates the "next" field in the graph state with routes to a node or finishes
        conditional_map = {k: k for k in members}
        conditional_map["FINISH"] = END
        workflow.add_conditional_edges("moderator", lambda x: x["next"], conditional_map)
        # Finally, add entrypoint
        workflow.set_entry_point("moderator")

        memory = SqliteSaver.from_conn_string(":memory:")
        graph = workflow.compile(checkpointer=memory)
        
        return graph
    
    def optimise_prompt(self):
        """
        Optimises a prompt by invoking a graph of expert agents.
        """
        # Initial state
        initial_state = {
            "messages": [HumanMessage(content=f"{self.base_prompt}", name="User")],
            "prompt": self.base_prompt,
            "next": "moderator"
        }

        # Construct the graph
        graph = self.construct_expert_graph()
        # display(Image(graph.get_graph().draw_mermaid_png()))

        config = {
            "configurable": {"thread_id": "1"},
            "recursion_limit": 50,
            }    

        # Run the graph
        for s in graph.stream(
            initial_state,
            config,
            stream_mode="values",
            ):
            if "__end__" not in s:
                s["messages"][-1].pretty_print()
                continue

        # if not os.path.exists("prompt_history_market.json"):
        #     with open("prompt_history_market.json", "w") as f:
        #         json.dump([], f)
        
        # with open("prompt_history_market.json", "r") as f:
        #     data = json.load(f)
        #     data.append(self.prompt_history)
            
        # with open("prompt_history_market.json", "w") as f:
        #     json.dump(data, f, indent=4)
        
        return s

In [7]:
# Prompt design team members
style_and_structure_principles = CorePrinciples([
    "Always structure prompts logically, for example COSTAR (Context, Objective, Steps, Task, Additional Information, Result)",
    "Always use a style and tone in prompts that is appropriate for the task",
    "Always consider the complexity of the task when designing prompts",
])
style_and_structure_expert = ExpertAgent("Style and Structure Expert", style_and_structure_principles)

conciseness_and_clarity_principles = CorePrinciples([
    "Always write clear and concise prompts",
    "Always use simple and direct language in prompts",
    "Always consider the task criteria when formatting and structuring prompts",
])
conciseness_and_clarity_expert = ExpertAgent("Conciseness and Clarity Expert", conciseness_and_clarity_principles)

contextual_relevance_principles = CorePrinciples([
    "Always provide context to help the model understand the task",
    "Always consider the context in which prompts will be used",
    "Always ensure personas and roles in prompts are optimal to the context of the task",
])
contextual_relevance_expert = ExpertAgent("Contextual Relevance Expert", contextual_relevance_principles)

task_alignment_principles = CorePrinciples([
    "Always ensure that prompts align with the task criteria",
    "Always tailor instructions to the task to guide the model",
])
task_alignment_expert = ExpertAgent("Task Alignment Expert", task_alignment_principles)

example_demonstration_principal = CorePrinciples([
    "Always provide examples to help the model understand the task",
    "Always ensure examples are relevant and clear",
    "Always demonstrate the expected output of the model",
])
example_demonstration_expert = ExpertAgent("Example Demonstration Expert", example_demonstration_principal)

avoiding_bias_principles = CorePrinciples([
    "Always avoid bias in prompts",
    "Always consider the ethical implications of prompts",
])
avoiding_bias_expert = ExpertAgent("Avoiding Bias Expert", avoiding_bias_principles)

incremental_prompting_principles = CorePrinciples([
    "Always provide clear step-by-step instructions to guide the model",
    "Always consider the complexity of the task when providing instructions",
])
incremental_prompting_expert = ExpertAgent("Incremental Prompting Expert", incremental_prompting_principles)

programming_logic_principles = CorePrinciples([
    "Only implement programming logic if you think it is beneficial for the task",
    "Structure prompts logically, similar to programming logic",
    "Implement programming logic including loops, conditionals, and functions and even pseudo-code",
])
programming_logic_expert = ExpertAgent("Programming Logic Expert", programming_logic_principles)

/Users/iwatson/Documents/Research Project/prompt-optimisation/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [8]:
# Domain team members
mathematics_principles = CorePrinciples([
    "Always follow good mathematical practices",
    "Always use mathetical operators correctly",
    "Always consider the mathematical principles relevant to the task",
])
mathematician = ExpertAgent("Mathematician", mathematics_principles)

word_problem_solving_principles = CorePrinciples([
    "Always pay attention to the keywords in problems",
    "Always approach problems systematically",
    "Always consider multiple approaches to solving problems",
])
word_problem_solver = ExpertAgent("Word Problem Solver", word_problem_solving_principles)


In [9]:
base_prompt = "`{content}`\nPlease output your answer at the end as ##<your answer (arabic numerals)>."
criteria = """- The prompt correctly instructs the LLM to solve a maths problem. 
- The prompt correctly outputs the answer at the end as ##<answer> with no spaces or units."""

leader_agent = ModeratorAgent(
    base_prompt=base_prompt,
    criteria=criteria,
    experts=[
        style_and_structure_expert,
        conciseness_and_clarity_expert,
        contextual_relevance_expert,
        task_alignment_expert,
        example_demonstration_expert,
        incremental_prompting_expert,
        programming_logic_expert,
        mathematician,
        word_problem_solver,
    ])
for expert in leader_agent.experts:
    print("Position: ", expert.position + "\nCore Principles: ", expert.core_principles)

Position:  Style and Structure Expert
Core Principles:  - Always structure prompts logically, for example COSTAR (Context, Objective, Steps, Task, Additional Information, Result)
- Always use a style and tone in prompts that is appropriate for the task
- Always consider the complexity of the task when designing prompts
Position:  Conciseness and Clarity Expert
Core Principles:  - Always write clear and concise prompts
- Always use simple and direct language in prompts
- Always consider the task criteria when formatting and structuring prompts
Position:  Contextual Relevance Expert
Core Principles:  - Always provide context to help the model understand the task
- Always consider the context in which prompts will be used
- Always ensure personas and roles in prompts are optimal to the context of the task
Position:  Task Alignment Expert
Core Principles:  - Always ensure that prompts align with the task criteria
- Always tailor instructions to the task to guide the model
Position:  Exampl

In [10]:
result = leader_agent.optimise_prompt()

================================ Human Message =================================
Name: User

`{content}`
Please output your answer at the end as ##<your answer (arabic numerals)>.


Position: Style and Structure Expert, Bid: 8
Position: Task Alignment Expert, Bid: 7
Position: Word Problem Solver, Bid: 5
Position: Programming Logic Expert, Bid: 5
Position: Incremental Prompting Expert, Bid: 5
Position: Conciseness and Clarity Expert, Bid: 5
Position: Example Demonstration Expert, Bid: 5
Position: Contextual Relevance Expert, Bid: 5
Position: Mathematician, Bid: 3
================================ Human Message =================================
Name: User

`{content}`
Please output your answer at the end as ##<your answer (arabic numerals)>.
Result: [HumanMessage(content='`{content}`\nPlease output your answer at the end as ##<your answer (arabic numerals)>.', id='2a8b24e3-6f1d-4d03-b96e-c7c90bd44c7d'), AIMessage(content='Using the provided {content}, please solve the maths problem. Output your answer at the end as ##<answer> with no spaces or units.', id='4043b2cd-941d-4fd8-983b-c7eb475ac0e3'), AIMessage(content='I made the prompt more explicit by specifying that th

In [11]:
result["prompt"]

"1. Read {content}, focusing on keywords. 2. Solve the math problem step-by-step, showing all intermediate steps clearly labeled and logically sequenced. For instance, if the problem is '2 + 2', the steps should show 'Step 1: 2 + 2 = 4'. 3. Output the final answer as ##4## with no spaces or units."

### Concurrent Runs

In [11]:
base_prompt = "Classify the sentence as positive or negative: {content}"
criteria = "This is a classification task with only two classes: positive and negative. The prompt should generalize."

moderator_agent_1 = ModeratorAgent(
    base_prompt=base_prompt,
    criteria=criteria,
)
for expert in moderator_agent_1.experts:
    print("Position: ", expert.position + "\nRole: ", expert.role + "\nFunction: ", expert.function + "\n")

moderator_agent_2 = ModeratorAgent(
    base_prompt=base_prompt,
    criteria=criteria,
)
for expert in moderator_agent_2.experts:
    print("Position: ", expert.position + "\nRole: ", expert.role + "\nFunction: ", expert.function + "\n")

moderator_agent_3 = ModeratorAgent(
    base_prompt=base_prompt,
    criteria=criteria,
)
for expert in moderator_agent_3.experts:
    print("Position: ", expert.position + "\nRole: ", expert.role + "\nFunction: ", expert.function + "\n")

Position:  Linguistic Analyst
Role:  The Linguistic Analyst is responsible for ensuring that the prompt is clear, concise, and unambiguous. They will analyze the language used in the prompt to ensure it is easily understandable and free of any potential biases.
Function:  The Linguistic Analyst will review and refine the wording of the prompt to ensure clarity and neutrality, preventing any language that could lead to misinterpretation or bias in classification.

Position:  Machine Learning Engineer
Role:  The Machine Learning Engineer is responsible for optimizing the prompt for machine learning models. They will ensure that the prompt is structured in a way that maximizes the model's ability to accurately classify the sentence as positive or negative.
Function:  The Machine Learning Engineer will test and iterate on the prompt structure to enhance the performance of classification algorithms, ensuring that the prompt facilitates accurate and reliable model outputs.

Position:  User E

KeyboardInterrupt: 

In [ ]:
# Assuming leader_agent is already defined and initialized
def run_optimisation(agent: LeaderAgent):
    return agent.optimise_prompt()

# Run 3 concurrent instances
with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
    futures = [executor.submit(run_optimisation, agent) for agent in [leader_agent_1, leader_agent_2, leader_agent_3]]
    results = [future.result() for future in concurrent.futures.as_completed(futures)]

for result in results:
    print(result)
    print("----")

class PromptMerge(BaseModel):
    """Merged prompt based on the best parts of each prompt."""
    final_prompt: str = Field(description="Result of merging prompts")

# OpenAI Agent to pull togther best parts of each result
def merge_results(results):
    """
    Agent to merge best parts of each prompt
    """
    llm = ChatOpenAI(
        temperature=1.0,
        model="gpt-4o",
    )
    system_message = """You are an experienced AI prompt engineer. Your role is to combine good prompts to create great prompts!
You have in-depth knowledge of large language models and prompt engineering best practices. Use knowledge at all times to guide your thinking."""

    template = """Your task is to merge the best parts of the prompts below to create a better prompt.
Carefully consider the strengths of each prompt and how they can be combined effectively whilst maintaining clarity and relevance.
You will be penalized if the prompt is repetitive, lacks clarity or is incoherent.
Aspects of the prompts to consider:
- Conciseness and clarity
- Contextual relevance
- Task alignment
- Example Demonstrations
- Avoiding bias
Consider aspects of good prompts beyond those listed above.
Placeholders are notated using curly braces. You must not remove placeholders or add additional placeholders.
I repeat, you must not remove placeholders or add additional placeholders.
Do not make assumptions on what the placeholders represent.

Prompts: {results}

Return only the next worker to process or 'FINISH' in JSON format below:

{{
    "final_prompt": "Result of merging prompts",
}}

{format_instructions}

You will be penalized if your output cannot be parsed correctly."""
    pydantic_parser = PydanticOutputParser(pydantic_object=PromptMerge)
    prompt_template = PromptTemplate(
        system_message=system_message,
        template=template,
        input_variables=["results"],
        partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
    )
    chain = prompt_template | llm | pydantic_parser
    for _ in range(3):
        try:
            output = chain.invoke({"results": results})
            break
        except Exception as e:
            print("Exception occurred:", e)
            continue
    return output.final_prompt

final_result = merge_results(results)

{'base_prompt': 'Classify the sentence as positive or negative: {content}', 'prompt_evolution': []}
{'base_prompt': 'Classify the sentence as positive or negative: {content}', 'prompt_evolution': [{'review': "The prompt should explicitly instruct the LLM to consider sentiment polarity, specify that the classes are only 'positive' or 'negative,' and ensure clarity in classification without introducing unnecessary complexities or ambiguities.", 'updated_prompt': "Classify the sentiment of the given sentence as either 'positive' or 'negative': {content}. Ensure the classification is based strictly on sentiment polarity."}]}
{'base_prompt': 'Classify the sentence as positive or negative: {content}', 'prompt_evolution': [{'review': 'The prompt needs clarity on the definition of positive and negative sentiment. Additionally, it should encourage consideration of context. Explicitly requiring the LLM to focus only on the sentiment will improve performance.', 'updated_prompt': 'Classify the sen

In [ ]:
print(final_result)

Classify the sentiment of the following sentence as either positive or negative: {content}. Evaluate tone and context carefully for precise classification. Ensure the classification is based strictly on sentiment polarity. Avoid neutrality. If sentiment is ambiguous, choose the more likely classification.
